In [17]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd

#Move indicator into its own Class so that will show up in output file. 
#Reference: https://www.backtrader.com/docu/inddev.html
'''
class RSI_Overbought(bt.Indicator):
    lines = ('overbought',)
    
    params = (
        ('level', 70),    
    )

    def __init__(self):
        self.lines.overbought = bt.indicators.RSI(self.data) > self.p.level
'''



class KST_Crossup(bt.Indicator):
    lines = ('crossup',)

    def __init__(self):
        self.lines.crossup = bt.indicators.KST(self.data) > 0

class KST_Crossdown(bt.Indicator):
    lines = ('crossdown',)

    def __init__(self):
        self.lines.crossdown = bt.indicators.KST(self.data) < 0
        
# Create a Stratey to trade hv
class HVStrategy(bt.Strategy):
    params = (
        ("symbols", None),
        ("max_days_in_trade", 7),
        ("y", None)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
#         print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
      
        # Keep a reference to the close price, historical vol as well as returns
        self.stock_px = self.datas[symbols.index("stockpx")].close
        self.vol = self.datas[symbols.index(y)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        self.flag = 0
        
        # Add indicators
        self.sma_short = bt.indicators.SimpleMovingAverage(self.stock_px, period=50)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.stock_px, period=200)
        self.dsma = self.sma_short - self.sma_long
        self.mom = bt.Cmp(self.sma_short, self.sma_long)
        self.rsi_overbought = bt.indicators.RSI(self.stock_px) > 70
        
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
        self.kst_crossup = KST_Crossup(self.stock_px)
        self.kst_crossdown = KST_Crossdown(self.stock_px)

        #self.rsi_oversold = bt.indicators.RSI(self.stock_px) <30

        self.cross_up = bt.indicators.CrossUp(self.sma_short, self.sma_long) 
        self.cross_down = bt.indicators.CrossDown(self.sma_short, self.sma_long)

        self.in_trade = False
        self.days_in_trade = 0
        
        
    def next(self):
        # Simply log the closing price of the series from the reference
#         self.log('Close, %.2f; Vol, %.2f' % (self.stock_px[0], self.vol[0]))
        
        size = self.position.size
        
        
#                 # If not in the trade
#         if (self.flag == 0):
#             if (self.kst_crossup[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
#                 #print("Breakout indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#                 self.flag = 1
                
#             if (self.kst_crossdown[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
#                 self.sell(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#                 self.flag = -1
            
            
#         # If in the long position
#         elif (self.flag == 1):
#             # Close the position if the SELL signal triggered or reached the max days to hold
#             if (self.kst_crossdown[0] > 0):
#                 self.close(exectype=bt.Order.Close)    # Close first, then sell
#                 self.days_in_trade = 0
#                # self.flag = 0
#                 self.sell(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#                 self.flag = -1
                
#             if (self.days_in_trade == self.p.max_days_in_trade):
#                 self.close(exectype=bt.Order.Close)
#                 self.days_in_trade = 0
#                 self.flag = 0
                    
#             elif (self.kst_crossup[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
#                 self.days_in_trade += 1
            
               # print("In position %d days" % self.days_in_trade)
        
#         # If in the short position
#         elif (self.flag == -1):
#             # Close the position if the BUY signal triggered or reached the max days to hold
#             if (self.kst_crossup[0] > 0):
#                 self.close(exectype=bt.Order.Close)
#                 #print("Closing the position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 0
#                 self.flag = 1
            
#             if (self.days_in_trade == self.p.max_days_in_trade):
#                 self.close(exectype=bt.Order.Close)
#                 #print("Closing the position")
#                 self.days_in_trade = 0
#                 self.flag = 0
                    
#             elif (self.kst_crossdown[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
#                 self.days_in_trade += 1
#                 #print("In position %d days" % self.days_in_trade)
                
#         #print("Flag = %d" % self.flag)
#        # print("In position %d days" % self.days_in_trade)
        
        
        
        
       #=============================================================================================== 
#         if (self.kst_crossdown[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):             
#             self.sell(exectype=bt.Order.Close,size=1000)
#             self.days_in_trade = 1
#             self.flag = 1
            

#         self.days_in_trade += 1
        
#         if (self.days_in_trade == self.p.max_days_in_trade):
#             self.close(exectype=bt.Order.Close)
#             self.days_in_trade = 0
        #===============================================================================================
        
        
        
        
        
        
        
        # Check if we are currently in a trade
        if(not size):
            #If not has our indicator triggered?
            if self.kst_crossup[0] > 0:
#                 print("Crossup indicator triggered...entering position")
                self.buy(exectype=bt.Order.Close)
                self.days_in_trade = 1
        else:
            self.days_in_trade += 1
#             print("In position %d days" % self.days_in_trade)
            if self.days_in_trade == self.p.max_days_in_trade:
#                 print("Max days to hold trade reached...closing position")
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
        '''      
        if(not size):
            #If not has our indicator triggered?
            if self.rsi_overbought[0] > 0:
                print("Overbought indicator triggered...entering position")
                self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
        else:
            self.days_in_trade += 1
            print("In position %d days" % self.days_in_trade)
            if self.days_in_trade == self.p.max_days_in_trade:
                print("Max days to hold trade reached...closing position")
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
        '''
                

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    # Add data
    datapath = os.path.join('../../../datas/YHOO.csv')
    
    datas = pd.read_csv(datapath, parse_dates=True,index_col=0)
    
    y = '10dORHV' #iv30
    symbol_cols = [y,'stockpx'] #symbol we're trying to forecast/trade first
    #symbol_cols = list(datas) #Load all symbols
    
    
    # Add the Data Feeds to Cerebro
    for i, symbol in enumerate(symbol_cols):
        data = datas[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add a strategy
    cerebro.addstrategy(HVStrategy, symbols=symbol_cols, y=y)

    # Add Analyzers
    cerebro.addanalyzer(bt.analyzers.TimeDrawDown,_name="DrawDown")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mySharpe',timeframe=7)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='TA')
    
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='HV-YHOO-KST-Crossup.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    thestrats = cerebro.run()
    thestrat = thestrats[0]


    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())  
    print('Sharpe Ratio:', thestrat.analyzers.mySharpe.get_analysis())
    print('Max Draw Down:',thestrat.analyzers.DrawDown.get_analysis(),"\n")
    #print('Trade Stats:',thestrat.analyzers.TA.get_analysis())
    for k,v in thestrat.analyzers.TA.get_analysis().items():
        print(k,":",v,"\n")

Starting Portfolio Value: 100000.00
Final Portfolio Value: 99956.42
Sharpe Ratio: OrderedDict([('sharperatio', -9.48453559024319)])
Max Draw Down: OrderedDict([('maxdrawdown', 0.12105612097821922), ('maxdrawdownperiod', 1389)]) 

total : AutoOrderedDict([('total', 171), ('open', 1), ('closed', 170)]) 

streak : AutoOrderedDict([('won', AutoOrderedDict([('current', 2), ('longest', 6)])), ('lost', AutoOrderedDict([('current', 0), ('longest', 6)]))]) 

pnl : AutoOrderedDict([('gross', AutoOrderedDict([('total', -31.940000000000023), ('average', -0.1878823529411766)])), ('net', AutoOrderedDict([('total', -31.940000000000023), ('average', -0.1878823529411766)]))]) 

won : AutoOrderedDict([('total', 86), ('pnl', AutoOrderedDict([('total', 553.59), ('average', 6.437093023255814), ('max', 49.98)]))]) 

lost : AutoOrderedDict([('total', 84), ('pnl', AutoOrderedDict([('total', -585.53), ('average', -6.970595238095238), ('max', -47.17999999999999)]))]) 

long : AutoOrderedDict([('total', 170), ('